In [50]:
## This is the notebook to subset the vcf file and subsequently one hot encode it

In [51]:
## Randomly subset the .vcf file

In [52]:
# requirements

import io
import os
import pandas as pd


In [53]:
# write a function that reads the vcf

def read_vcf(path):
    with open(path, 'r') as f:
        lines = [l for l in f if not l.startswith('##')]
    return pd.read_csv(
        io.StringIO(''.join(lines)),
        dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
               'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t'
    ).rename(columns={'#CHROM': 'CHROM'})

In [54]:
# define the path for the vcf

vcf = ("/Users/madisoncreach/CSS_844/Module2/g2f_mi_2020-2023_LD.vcf")

In [55]:
# use the function to read the vcf

vcf_file = read_vcf(vcf)

In [56]:
print(vcf_file)

      CHROM        POS             ID REF ALT QUAL FILTER INFO FORMAT  \
0         1     660286      S1_660286   T   C    .   PASS    .     GT   
1         1     680802      S1_680802   A   C    .   PASS    .     GT   
2         1     681760      S1_681760   C   T    .   PASS    .     GT   
3         1     684410      S1_684410   A   G    .   PASS    .     GT   
4         1     684782      S1_684782   C   T    .   PASS    .     GT   
...     ...        ...            ...  ..  ..  ...    ...  ...    ...   
54838    10  151947444  S10_151947444   G   A    .   PASS    .     GT   
54839    10  152010224  S10_152010224   A   T    .   PASS    .     GT   
54840    10  152014369  S10_152014369   C   T    .   PASS    .     GT   
54841    10  152267521  S10_152267521   A   G    .   PASS    .     GT   
54842    10  152269703  S10_152269703   A   G    .   PASS    .     GT   

      B14A/H95  ... W10010_0310/LH244 W10010_0311/LH244 W10010_0337/LH244  \
0          1/1  ...               0/0         

In [57]:
# randomly subset the vcf file to 10,000 lines

subset_vcf = vcf_file.sample(n=10000, random_state=42)



In [58]:
print(subset_vcf)

      CHROM        POS            ID REF ALT QUAL FILTER INFO FORMAT B14A/H95  \
33517     5  216898908  S5_216898908   C   T    .   PASS    .     GT      0/0   
3597      1  160878820  S1_160878820   T   G    .   PASS    .     GT      0/0   
25777     4  197236918  S4_197236918   A   G    .   PASS    .     GT      1/0   
24615     4  157980321  S4_157980321   C   G    .   PASS    .     GT      1/1   
35722     6  117819854  S6_117819854   G   T    .   PASS    .     GT      1/0   
...     ...        ...           ...  ..  ..  ...    ...  ...    ...      ...   
9532      2   43274946   S2_43274946   G   A    .   PASS    .     GT      0/0   
41503     8    1133835    S8_1133835   C   A    .   PASS    .     GT      0/0   
41782     8    5303736    S8_5303736   G   C    .   PASS    .     GT      0/1   
18794     3  178519029  S3_178519029   C   T    .   PASS    .     GT      0/0   
34798     6   51234469   S6_51234469   G   A    .   PASS    .     GT      0/0   

       ... W10010_0310/LH24

In [59]:
# tranpose the dataframe for downstream merging purposes

vcf_t = subset_vcf.T

In [60]:
print(vcf_t)

                          33517         3597          25777         24615  \
CHROM                         5             1             4             4   
POS                   216898908     160878820     197236918     157980321   
ID                 S5_216898908  S1_160878820  S4_197236918  S4_157980321   
REF                           C             T             A             C   
ALT                           T             G             G             G   
...                         ...           ...           ...           ...   
W10010_0366/LH244           1/0           1/0           1/1           0/0   
W10010_0381/LH244           0/0           0/0           1/1           1/0   
W10010_0452/LH244           0/0           0/0           1/1           1/0   
W10010_0464/LH244           1/0           1/0           1/1           0/0   
W22/LH244                   0/0           0/0           1/1           1/0   

                          35722       27583        28407          53641  \


In [61]:
# get rid of chrom, pos, ID, and other extraneous information (this is dumb but it works)

vcf_clean1 = vcf_t.iloc[2:]
vcf_clean2 = vcf_clean1.drop("REF")
vcf_clean3 = vcf_clean2.drop("ALT")
vcf_clean4 = vcf_clean3.drop("QUAL")
vcf_clean5 = vcf_clean4.drop("FILTER")
vcf_clean6 = vcf_clean5.drop("INFO")
vcf_clean_final = vcf_clean6.drop("FORMAT")
print(vcf_clean_final)

                          33517         3597          25777         24615  \
ID                 S5_216898908  S1_160878820  S4_197236918  S4_157980321   
B14A/H95                    0/0           0/0           1/0           1/1   
B14A/MO17                   0/0           0/0           1/1           1/0   
B14A/OH43                   0/0           0/1           1/1           1/1   
B37/H95                     0/0           0/0           0/0           1/1   
...                         ...           ...           ...           ...   
W10010_0366/LH244           1/0           1/0           1/1           0/0   
W10010_0381/LH244           0/0           0/0           1/1           1/0   
W10010_0452/LH244           0/0           0/0           1/1           1/0   
W10010_0464/LH244           1/0           1/0           1/1           0/0   
W22/LH244                   0/0           0/0           1/1           1/0   

                          35722       27583        28407          53641  \


In [62]:
# next one hot encode all the snps

vcf_replace = vcf_clean_final.replace("0/0", -1)
vcf_replace2 = vcf_replace.replace("0/1", 0)
vcf_replace3 = vcf_replace2.replace("1/0", 0)
vcf_replace_final = vcf_replace3.replace("1/1", 1)

In [63]:
print(vcf_replace_final)

                          33517         3597          25777         24615  \
ID                 S5_216898908  S1_160878820  S4_197236918  S4_157980321   
B14A/H95                     -1            -1             0             1   
B14A/MO17                    -1            -1             1             0   
B14A/OH43                    -1             0             1             1   
B37/H95                      -1            -1            -1             1   
...                         ...           ...           ...           ...   
W10010_0366/LH244             0             0             1            -1   
W10010_0381/LH244            -1            -1             1             0   
W10010_0452/LH244            -1            -1             1             0   
W10010_0464/LH244             0             0             1            -1   
W22/LH244                    -1            -1             1             0   

                          35722       27583        28407          53641  \


In [64]:
# put the final cleaned vcf in a csv
#vcf_replace_final.to_csv("/Users/madisoncreach/CSS_844/Module2/g2f_final_vcf.csv", header=True, index=True)

In [65]:
# read in 2nd vcf 
vcf_df2 = ('/Users/madisoncreach/CSS_844/Module2/2020-2021genotypes.vcf')

vcf_file2 = read_vcf(vcf_df2)

In [66]:
print(vcf_file2)

       CHROM        POS             ID REF ALT QUAL FILTER INFO FORMAT  \
0          1     121343      S1_121343   C   T    .   PASS    .     GT   
1          1     122279      S1_122279   C   T    .   PASS    .     GT   
2          1     122508      S1_122508   A   G    .   PASS    .     GT   
3          1     122872      S1_122872   T   A    .   PASS    .     GT   
4          1     126019      S1_126019   A   G    .   PASS    .     GT   
...      ...        ...            ...  ..  ..  ...    ...  ...    ...   
325940    10  152269703  S10_152269703   A   G    .   PASS    .     GT   
325941    10  152271549  S10_152271549   C   T    .   PASS    .     GT   
325942    10  152279554  S10_152279554   G   A    .   PASS    .     GT   
325943    10  152282265  S10_152282265   G   A    .   PASS    .     GT   
325944    10  152284989  S10_152284989   T   C    .   PASS    .     GT   

       F42/OH43  ... W10004_0065/PHP02 W10004_0424/PHP02 W10004_0018/PHP02  \
0           0/0  ...             

In [67]:
# do everything to the 2nd vcf that was done to the first

vcf2_t = vcf_file2.T

vcf2_clean1 = vcf2_t.iloc[2:]
vcf2_clean2 = vcf2_clean1.drop("REF")
vcf2_clean3 = vcf2_clean2.drop("ALT")
vcf2_clean4 = vcf2_clean3.drop("QUAL")
vcf2_clean5 = vcf2_clean4.drop("FILTER")
vcf2_clean6 = vcf2_clean5.drop("INFO")
vcf2_clean_final = vcf2_clean6.drop("FORMAT")
print(vcf2_clean_final)

vcf2_replace = vcf2_clean_final.replace("0/0", -1)
vcf2_replace2 = vcf2_replace.replace("0/1", 0)
vcf2_replace3 = vcf2_replace2.replace("1/0", 0)
vcf2_replace_final = vcf2_replace3.replace("1/1", 1)

                      0          1          2          3          4       \
ID                 S1_121343  S1_122279  S1_122508  S1_122872  S1_126019   
F42/OH43                 0/0        0/0        0/0        0/0        0/0   
B73/MO17                 0/0        0/0        0/1        0/1        0/1   
CG119/CG108              0/0        0/0        0/0        0/0        0/0   
TX777/LH195              0/0        0/0        0/0        0/0        0/0   
...                      ...        ...        ...        ...        ...   
W10004_1037/PHP02        1/1        1/1        0/0        0/0        0/0   
W10004_0991/PHP02        1/0        1/0        0/0        0/0        0/0   
W10004_0134/PHP02        0/1        0/1        0/0        0/0        0/0   
W10004_0988/PHP02        1/0        1/0        0/0        0/0        0/0   
W10004_0006/PHP02        0/1        0/1        0/0        0/0        0/0   

                      5          6          7          8          9       ...  \
ID    

In [68]:
print(vcf2_replace_final)

                      0          1          2          3          4       \
ID                 S1_121343  S1_122279  S1_122508  S1_122872  S1_126019   
F42/OH43                  -1         -1         -1         -1         -1   
B73/MO17                  -1         -1          0          0          0   
CG119/CG108               -1         -1         -1         -1         -1   
TX777/LH195               -1         -1         -1         -1         -1   
...                      ...        ...        ...        ...        ...   
W10004_1037/PHP02          1          1         -1         -1         -1   
W10004_0991/PHP02          0          0         -1         -1         -1   
W10004_0134/PHP02          0          0         -1         -1         -1   
W10004_0988/PHP02          0          0         -1         -1         -1   
W10004_0006/PHP02          0          0         -1         -1         -1   

                      5          6          7          8          9       ...  \
ID    

In [69]:
# set the ID row as header in both dfs

new_header1 = vcf_replace_final.iloc[0] 
vcf_replace_final = vcf_replace_final[1:] 
vcf_replace_final.columns = new_header1

In [70]:
# set the ID row as header in 2nd

new_header2 = vcf2_replace_final.iloc[0] 
vcf2_replace_final = vcf2_replace_final[1:] 
vcf2_replace_final.columns = new_header2

In [71]:
print(vcf_replace_final)

ID                S5_216898908 S1_160878820 S4_197236918 S4_157980321  \
B14A/H95                    -1           -1            0            1   
B14A/MO17                   -1           -1            1            0   
B14A/OH43                   -1            0            1            1   
B37/H95                     -1           -1           -1            1   
B37/MO17                    -1           -1            0            0   
...                        ...          ...          ...          ...   
W10010_0366/LH244            0            0            1           -1   
W10010_0381/LH244           -1           -1            1            0   
W10010_0452/LH244           -1           -1            1            0   
W10010_0464/LH244            0            0            1           -1   
W22/LH244                   -1           -1            1            0   

ID                S6_117819854 S5_5316857 S5_15251077 S10_129000411  \
B14A/H95                     0         -1          -

In [72]:
print(vcf_replace_final.columns)

Index(['S5_216898908', 'S1_160878820', 'S4_197236918', 'S4_157980321',
       'S6_117819854', 'S5_5316857', 'S5_15251077', 'S10_129000411',
       'S8_4088046', 'S10_90208442',
       ...
       'S5_222925253', 'S2_27952207', 'S5_145525481', 'S5_91532008',
       'S3_127623411', 'S2_43274946', 'S8_1133835', 'S8_5303736',
       'S3_178519029', 'S6_51234469'],
      dtype='object', name='ID', length=10000)


In [73]:
concatenated = pd.concat([vcf_replace_final, vcf2_replace_final], join="inner")

In [76]:
print(concatenated.shape)

(1292, 10000)


In [75]:
#concatenated.to_csv("/Users/madisoncreach/CSS_844/Module2/2020-2023_vcf_final.csv", header=True, index=True)